In [4]:
# 합성곱 계층 구현 by im2col
# im2col은 만들어진 것을 사용
import numpy as np

import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7) # (데이터 수, 채널, 높이, 너비)
col1 = im2col(x1,5,5,stride = 1, pad = 0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride = 1, pad = 0)
print(col2.shape) # 75는 필터의 원소수와 같다(채널 3개 * 5*5)
                  # 90은 데이터 수가 10배( 9 * 10(데이터 수))

(9, 75)
(90, 75)


In [5]:
# 합성곱 계층 구현 by im2col
class Convolution:
    def __init__(self, W, b, stride = 1, pad = 0): # 필터(가중치), 편향, 스트라이드, 패딩
        self.W = W
        slef.b = b
        self.stride = stride
        self.pad = pad
    
    def forword(self,x):
        FN, C, FH, FW = self.W.shape # FN:필터개수, C:채널, FH,FW : 채널 높이,너비
        N, C, H, W = x.shape # 
        out_h = int(1+(H + 2*self.pad - FH) / self.stride)
        out_w = int(1+(W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad) #입력데이터 im2col로 전개
        col_W = self.W.reshape(FN, -1).T #필터reshpe ; -1을 하면 다차원 배열의 원소 수가 변환 후에도 똑같이 유지되도록 적절히 묶어줌
        out = np.dot(col,col_W)+ self.b  #행렬곱+편향
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2) # 출력데이터를 적절한 형상으로 바꿈;transpose:축순서변경
        
        return out


In [7]:
#풀링계층 구현 
class Pooling:
    def __init__(self, pool_h, pool_w, stride = 1, pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forword(self,x):
        N, C, H, W = x.shape
        out_h = int(1+(H - self.pool_h) / self.stride)
        out_w = int(1+(W - self.pool_w) / self.stride)
        
        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1,slef.pool_h*self.pool_w)
        
        # 최댓값 (2)
        out = np.max(col, axis =1)
        
        # 성형 (3)
        out = out.reshape(N,out_h, out_w, C).transpose(0,3,1,2)
        
        return out

In [1]:
# CNN 구현 : 합성곱과 풀링 계층 조합
class SimpleConvNet:
    def __init__(self, input_dim = (1,28,28),
                conv_param = {'filter_num ':30, 'filter_size':5,
                             'pad':0, 'stride':1},
                hidden_size = 100, output_size = 10, weight_init_std = 0.01):
        # conv_param을 딕셔너리에서 꺼냄
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        #합성곱 계층의 출력 크기 계산
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride +1
        pool_output_size = int(filter_num*(conv_output_size/2)*(conv_output_size/2))
        
        #가중치 매개변수 초기화
        self.params = {} # 1번째 층은 합성곱 계층 / 나머지 두개는 완전연결 계층
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0].filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size,hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b1'] = np.zeros(output_size)
        
        #CNN 구성 계층 생성
        self.layers = OrderDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_params['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2,stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    
    # 추론
    def predict(self,x):
        for layer in self.layers.values():
            x= layer.forward(x)
        return x
    
    #손실
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y,t)
                               
    # 오차역전파법
    def gradient(self,x,t):
        #순전파 
        self.loss(x,t)
        
        #역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        #결과 저장
        grads ={}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv2'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads